# Apache Hudi Core Conceptions (3) - MOR: File Layouts + Compaction

## 1. Configuration

In [1]:
%%sh
# deploy hudi bundle jar
hdfs dfs -copyFromLocal -f /usr/lib/hudi/hudi-spark-bundle.jar /tmp/hudi-spark-bundle.jar
hdfs dfs -ls /tmp/hudi-spark-bundle.jar

-rw-r--r--   1 emr-notebook hdfsadmingroup   61421977 2023-03-10 12:54 /tmp/hudi-spark-bundle.jar


In [2]:
%%configure -f
{
    "conf" : {
        "spark.jars":"hdfs:///tmp/hudi-spark-bundle.jar",            
        "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
        "spark.sql.extensions":"org.apache.spark.sql.hudi.HoodieSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.hudi.catalog.HoodieCatalog"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
51,application_1678096020253_0074,spark,idle,Link,Link,None,
53,application_1678096020253_0079,spark,idle,Link,Link,None,


In [3]:
%env S3_BUCKET=apache-hudi-core-conceptions

env: S3_BUCKET=apache-hudi-core-conceptions


In [4]:
%%sql
set S3_BUCKET=apache-hudi-core-conceptions

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
55,application_1678096020253_0084,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [5]:
%env WORKSPACE=/home/emr-notebook/apache-hudi-core-conceptions

env: WORKSPACE=/home/emr-notebook/apache-hudi-core-conceptions


In [6]:
%%sh
# make workspace
mkdir -p $WORKSPACE
# deploy hudi-stat.sh, a utility shell script to output hudi table status
wget https://raw.githubusercontent.com/bluishglc/apache-hudi-core-conceptions/master/hudi-stat.sh -O $WORKSPACE/hudi-stat.sh &>/dev/null
chmod a+x $WORKSPACE/hudi-stat.sh
ls $WORKSPACE/hudi-stat.sh

/home/emr-notebook/apache-hudi-core-conceptions/hudi-stat.sh


In [7]:
%%html
<style>
table {float:left}
</style>

## 2. Test Case 1 - Sync Compaction: Inline Schedule + Inline Execute

### 2.1. Test Plan

Step No.|Action|Volume Per Partition |Storage
:--------:|:------|:------|:----------
1|Insert|41MB|+1 Base File
2|Update|177KB|+1 Log File
3|Update|252KB|+1 Log File, +1 Base File
4|Update|258KB|+1 Log File
5|Update|280KB|+1 Log File

### 2.2. Key Settings

KEY|DEFAULT VALUE|SET VALUE
:---|:---|:---
hoodie.compact.inline|false|true
hoodie.compact.schedule.inline|false|Default Value
hoodie.compact.inline.max.delta.commits|5|3
hoodie.copyonwrite.record.size.estimate|1024|175

### 2.3. Set Variables

In [8]:
%%sql
set TABLE_NAME=reviews_mor_1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%env TABLE_NAME=reviews_mor_1

env: TABLE_NAME=reviews_mor_1


### 2.4. Create Table

In [10]:
%%sh
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ${WORKSPACE}/${TABLE_NAME}
sleep 5

In [11]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
%%sql
drop table if exists ${TABLE_NAME}_ro

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [13]:
%%sql
drop table if exists ${TABLE_NAME}_rt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
%%sql
create table if not exists ${TABLE_NAME} (
    review_id string, 
    star_rating long, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'mor',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.copyonwrite.record.size.estimate = '175',
    hoodie.compact.inline = 'true',
    -- hoodie.compact.schedule.inline = 'false',
    hoodie.compact.inline.max.delta.commits = '3'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [15]:
%%sql
set YEAR=1999

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [16]:
%%sql
select
    l.*, l.records_num / r.total as percentage
from
    (select star_rating, count(1) as records_num from reviews where year=${YEAR} group by star_rating) as l
join
    (select count(1) as total from reviews where year=${YEAR}) as r
order by 
    star_rating asc;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 2.5. Insert 41MB / +1 Base File

In [17]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 1999

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [18]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310125522704 │ deltacommit │ COMPLETED │ 03-10 12:55 │ 03-10 12:55 │ 03-10 12:55 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪════════

### 2.6. Update 177KB / +1 Log File

In [19]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '1999-01-01'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [20]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310125522704 │ deltacommit │ COMPLETED │ 03-10 12:55 │ 03-10 12:55 │ 03-10 12:55 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310125624734 │ deltacommit │ COMPLETED │ 03-10 12:56 │ 03-10 12:56 │ 03-10 12:56 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        

### 2.7. Update 252KB / +1 Log File +1 Base File

In [21]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '1999-01-02'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [22]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310125522704 │ deltacommit │ COMPLETED │ 03-10 12:55 │ 03-10 12:55 │ 03-10 12:55 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310125624734 │ deltacommit │ COMPLETED │ 03-10 12:56 │ 03-10 12:56 │ 03-10 12:56 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230310125706103 │ deltacommit │ COMPLETED │ 03-10 12:57 │ 03-10 12:57 │ 03-10 12:57 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230310125720044

### 2.8. Update 258KB / +1 Log File

In [23]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '1999-01-03'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [24]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310125522704 │ deltacommit │ COMPLETED │ 03-10 12:55 │ 03-10 12:55 │ 03-10 12:55 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310125624734 │ deltacommit │ COMPLETED │ 03-10 12:56 │ 03-10 12:56 │ 03-10 12:56 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230310125706103 │ deltacommit │ COMPLETED │ 03-10 12:57 │ 03-10 12:57 │ 03-10 12:57 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230310125720044

### 2.9. Update 280KB / +1 Log File

In [25]:
%%sql
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '1999-01-04'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [26]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310125522704 │ deltacommit │ COMPLETED │ 03-10 12:55 │ 03-10 12:55 │ 03-10 12:55 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310125624734 │ deltacommit │ COMPLETED │ 03-10 12:56 │ 03-10 12:56 │ 03-10 12:56 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230310125706103 │ deltacommit │ COMPLETED │ 03-10 12:57 │ 03-10 12:57 │ 03-10 12:57 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230310125720044

## 3. Test Case 2 - Async Compaction: Inline Schedule + Offline Execute

### 3.1. Create Table

In [27]:
%%sql
set TABLE_NAME=reviews_mor_2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [28]:
%%sql
set TABLE_PATH=s3://glc-examples/hudi-core-conceptions/reviews_mor_2

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [29]:
%env TABLE_NAME=reviews_mor_2

env: TABLE_NAME=reviews_mor_2


In [30]:
%env TABLE_PATH=s3://glc-examples/hudi-core-conceptions/reviews_mor_2

env: TABLE_PATH=s3://glc-examples/hudi-core-conceptions/reviews_mor_2


In [31]:
%%sh
echo $(basename s3://${S3_BUCKET}/${TABLE_NAME})
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ~/${TABLE_NAME}
sleep 3

reviews_mor_2


In [32]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [33]:
%%sql
drop table if exists ${TABLE_NAME}_ro

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [34]:
%%sql
drop table if exists ${TABLE_NAME}_rt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [35]:
%%sql
create table if not exists ${TABLE_NAME} (
    review_id string, 
    star_rating long, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'mor',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.copyonwrite.record.size.estimate = '175',
    hoodie.compact.inline = 'false',
    hoodie.compact.schedule.inline = 'true',
    hoodie.compact.inline.max.delta.commits = '3'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 3.2. Batch 1 - Insert

In [36]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 1999

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [37]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310125937816 │ deltacommit │ COMPLETED │ 03-10 12:59 │ 03-10 12:59 │ 03-10 13:00 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪════════

### 3.3. Batch 2 - Update

In [38]:
%%sql
-- 更新1999-01-01的评价
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '1999-01-01'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [39]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310125937816 │ deltacommit │ COMPLETED │ 03-10 12:59 │ 03-10 12:59 │ 03-10 13:00 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310130033905 │ deltacommit │ COMPLETED │ 03-10 13:00 │ 03-10 13:00 │ 03-10 13:00 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        

### 3.4. Batch 3 - Update

In [40]:
%%sql
-- 更新1999-01-02的评价
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '1999-01-02'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [41]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310125937816 │ deltacommit │ COMPLETED │ 03-10 12:59 │ 03-10 12:59 │ 03-10 13:00 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310130033905 │ deltacommit │ COMPLETED │ 03-10 13:00 │ 03-10 13:00 │ 03-10 13:00 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230310130112519 │ deltacommit │ COMPLETED │ 03-10 13:01 │ 03-10 13:01 │ 03-10 13:01 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230310130126770

### 3.5. Async Execute Compaction

In [42]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class "org.apache.hudi.utilities.HoodieCompactor" \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'execute' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "$TABLE_NAME" > ~/${TABLE_NAME}.compaction.execute.out &>/dev/null

In [43]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310125937816 │ deltacommit │ COMPLETED │ 03-10 12:59 │ 03-10 12:59 │ 03-10 13:00 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310130033905 │ deltacommit │ COMPLETED │ 03-10 13:00 │ 03-10 13:00 │ 03-10 13:00 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230310130112519 │ deltacommit │ COMPLETED │ 03-10 13:01 │ 03-10 13:01 │ 03-10 13:01 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230310130126770

## 4. Test Case 3 - Async Compaction: Offline Schedule + Offline Execute

### 4.1. Create Table

In [44]:
%%sql
set TABLE_NAME=reviews_mor_3

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [45]:
%%sql
set TABLE_PATH=s3://glc-examples/hudi-core-conceptions/reviews_mor_3

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [46]:
%env TABLE_NAME=reviews_mor_3

env: TABLE_NAME=reviews_mor_3


In [47]:
%env TABLE_PATH=s3://glc-examples/hudi-core-conceptions/reviews_mor_3

env: TABLE_PATH=s3://glc-examples/hudi-core-conceptions/reviews_mor_3


In [48]:
%%sh
echo $(basename s3://${S3_BUCKET}/${TABLE_NAME})
aws s3 rm s3://${S3_BUCKET}/${TABLE_NAME} --recursive &>/dev/null
rm -rf ~/${TABLE_NAME}
sleep 3

reviews_mor_3


In [49]:
%%sql
drop table if exists ${TABLE_NAME}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [50]:
%%sql
drop table if exists ${TABLE_NAME}_ro

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [51]:
%%sql
drop table if exists ${TABLE_NAME}_rt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [52]:
%%sql
create table if not exists ${TABLE_NAME} (
    review_id string, 
    star_rating long, 
    review_body string, 
    review_date date, 
    year long,
    timestamp long,
    parity int
)
using hudi
location 's3://${S3_BUCKET}/${TABLE_NAME}'
partitioned by (parity)
options ( 
    type = 'mor',  
    primaryKey = 'review_id', 
    preCombineField = 'timestamp',
    hoodie.copyonwrite.record.size.estimate = '175',
    -- hoodie.compact.inline = 'false',
    -- hoodie.compact.schedule.inline = 'false',
    hoodie.compact.inline.max.delta.commits = '3'
);

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### 4.2. Batch 1 - Insert

In [53]:
%%sql
insert into 
    ${TABLE_NAME}
select 
    review_id, 
    star_rating, 
    review_body, 
    review_date, 
    year,
    unix_timestamp(current_timestamp()) as timestamp,
    mod(crc32(review_id), 2) as parity
from
    reviews
where
    year = 1999

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [54]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310130318582 │ deltacommit │ COMPLETED │ 03-10 13:03 │ 03-10 13:03 │ 03-10 13:04 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠═══════════════════╪════════

### 4.3. Batch 2 - Update

In [55]:
%%sql
-- 更新1999-01-01的评价
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    review_date = '1999-01-01'
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [56]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310130318582 │ deltacommit │ COMPLETED │ 03-10 13:03 │ 03-10 13:03 │ 03-10 13:04 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310130428939 │ deltacommit │ COMPLETED │ 03-10 13:04 │ 03-10 13:04 │ 03-10 13:04 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔═══════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime        

### 4.4. Batch 3 - Update

In [57]:
%%sql
-- 更新1999-01-02的评价
update
    ${TABLE_NAME}
set             
    review_body = concat(uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid(),uuid()),
    timestamp = unix_timestamp(current_timestamp())
where
    star_rating = 5
;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [58]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310130318582 │ deltacommit │ COMPLETED │ 03-10 13:03 │ 03-10 13:03 │ 03-10 13:04 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310130428939 │ deltacommit │ COMPLETED │ 03-10 13:04 │ 03-10 13:04 │ 03-10 13:04 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230310130509649 │ deltacommit │ COMPLETED │ 03-10 13:05 │ 03-10 13:05 │ 03-10 13:05 ║
╚═════╧═══════════════════╧═════════════╧═══════════╧═════════════╧═════════════╧═════════════╝

[ COMMITS ]

╔══════════

### 4.5. Async Schedule Compaction

In [59]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class 'org.apache.hudi.utilities.HoodieCompactor' \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'schedule' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "$TABLE_NAME" \
  --hoodie-conf "hoodie.compact.inline.max.delta.commits=3" > ~/${TABLE_NAME}.compaction.schedule.out &>/dev/null

In [60]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310130318582 │ deltacommit │ COMPLETED │ 03-10 13:03 │ 03-10 13:03 │ 03-10 13:04 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310130428939 │ deltacommit │ COMPLETED │ 03-10 13:04 │ 03-10 13:04 │ 03-10 13:04 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230310130509649 │ deltacommit │ COMPLETED │ 03-10 13:05 │ 03-10 13:05 │ 03-10 13:05 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230310130622297

### 4.6. Async Execute Compaction

In [61]:
%%sh
# it's required for current user (emr-notebook) to get sudo permission
sudo -u hadoop spark-submit \
  --jars '/usr/lib/hudi/hudi-spark-bundle.jar' \
  --class "org.apache.hudi.utilities.HoodieCompactor" \
  /usr/lib/hudi/hudi-utilities-bundle.jar \
  --spark-memory '4g' \
  --mode 'execute' \
  --base-path "s3://${S3_BUCKET}/${TABLE_NAME}" \
  --table-name "$TABLE_NAME" > ~/${TABLE_NAME}.compaction.execute.out &>/dev/null

In [62]:
%%sh
~/hudi-stat.sh s3://${S3_BUCKET}/${TABLE_NAME} timeline commits compactions storage


[ TIMELINE ]

╔═════╤═══════════════════╤═════════════╤═══════════╤═════════════╤═════════════╤═════════════╗
║ No. │ Instant           │ Action      │ State     │ Requested   │ Inflight    │ Completed   ║
║     │                   │             │           │ Time        │ Time        │ Time        ║
╠═════╪═══════════════════╪═════════════╪═══════════╪═════════════╪═════════════╪═════════════╣
║ 0   │ 20230310130318582 │ deltacommit │ COMPLETED │ 03-10 13:03 │ 03-10 13:03 │ 03-10 13:04 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 1   │ 20230310130428939 │ deltacommit │ COMPLETED │ 03-10 13:04 │ 03-10 13:04 │ 03-10 13:04 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 2   │ 20230310130509649 │ deltacommit │ COMPLETED │ 03-10 13:05 │ 03-10 13:05 │ 03-10 13:05 ║
╟─────┼───────────────────┼─────────────┼───────────┼─────────────┼─────────────┼─────────────╢
║ 3   │ 20230310130622297